### Madden Ultimate Team (MUT) Auction House Analyzer

A set in madden is where you buy n specific cards to complete the set and recieve a more valuable player in return.
Given an input of set players, ids, and the reward allows you to find arbitrage in the MUT Auction House.

Currently:
- Currently running two parallel arrays of set players and their ids
- Keep track of target player and target_id 
- Use selenium to open webdriver Firefox
- Manually log into 'Muthead' (Must be logged in to refresh auction prices)
- Selenium refreshes all of the auction prices of relevant players page by page
- Use beautifulsoup to scrape from three tables: Current Auctions, Completed Sales, and 'Stats for Nerds'
- Using the minimum BuyNow price for each player, the price to complete the set is calculated
- Using this, an average of last n sales of player and taking into account the 10% tax fee on the Auction House a conditional is triggered to pursue the set based on expected profit

Current Pitfalls:
- Need to manually login
- Poor flow
- Likely needs proper use of .sleep() 

Planned:
- store data, pickle?
- to also use for non-set (rare) players


In [90]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import requests

## methods

In [91]:
def retrieve_prices(auction_table):    # method that returns a dataframe of current auctions
    
    #print (soup.title.text) # print player page title
    
    rows = auction_table.find_all('tr')
    num_rows = min ( len(rows), 5 )
    #title row
    row = rows[0]
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 

    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)

    new_table = pd.DataFrame(columns=titles, index=range(0,num_rows-1))

    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
    
        #time remaining
        time_rem = values[0][58:67].replace("<", "").replace(">", "")
        num_bids = values[1]
        curr_bid = values[2]
        buy_now = values[3]
        nums = [time_rem, num_bids, curr_bid, buy_now]
        j=0
    
        for title in titles:
            new_table[title][i-1] = nums[j]
            j += 1
    
    return new_table

In [92]:
def retrieve_stats(hist_table):  # method to return dataframe of stats for last 100 sales such as quartile prices, min, avg
    
    rows = hist_table.find_all('tr')

    new_table2 = pd.DataFrame(columns=['Q', 'Price'], index=[0,1,2,3,4,5,6])
    
    for i in range(0,7): #first row of prices
        row = rows[i]
        td_tags = row.find_all('td')
        
        values = []

        for val in td_tags:
            values.append(str(val))
 
        #time remaining
        ca = values[0]
        cb = values[1]
    
        nums = [ca,cb]
        new_table2['Q'][i] = ca
        new_table2['Price'][i] = cb
    
    return new_table2

In [93]:
def retrieve_sales(sales_table):    # method to return dataframe of last n completed sales ( <= 10?)
    
    rows = sales_table.find_all('tr')
    num_rows = min ( len(rows), 10 )
    
    row = rows[0] #title row
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 
    
    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)
    new_table3 = pd.DataFrame(columns=titles, index=range(0,num_rows-1))
   
    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
            
        time_sold = values[0][119:140].replace("<", "").replace(">", "")
        price_sold = values[1].replace(".", "").replace(",", "")
        
        nums = [time_sold, price_sold] #print(nums) # debug
        j=0
    
        for title in titles:
            new_table3[title][i-1] = nums[j]
            j += 1
            
    new_table3['Price'] = pd.to_numeric( new_table3['Price'] )
    return new_table3

## price refresh

In [43]:
# RUN THIS

browser = webdriver.Firefox()

URL = 'https://www.muthead.com/twitch-login?returnUrl=%2f'

browser.get(URL)

#manually log in

# data frame to store scraped data
data = pd.DataFrame(columns=['Name', 'Min Price', 'Average', 'Date'], index=[0])

# start here
PLAYERS WE WANT TO SEARCH ARE DIVIDED INTO TWO LISTS OF NAMES AND PLAYERIDS


In [106]:
player_name_list = []
player_id_list = []
target_list = []
target_id_list = []

# Set Player Names
player_name_list.append( ['Andy Dalton', 'Laremy Tunsil', 'Lamer Miller', 'Eric Decker', 'Ben Watson', \
                         'Spencer Long', 'Brian Lafell', 'Don Barclay'] )
#player_name_list.append( ['(95) Ed Reed', '(94) Ed Reed', '(92) Ed Reed', '(91) Ed Reed', '(89) Ed Reed'] )

# Set Player IDs
player_id_list.append ( ['34093', '34094', '34095', '34096', '34101', '34102', '34103', '34104'] )
#player_id_list.append ( ['32178', '32179', '32180', '32181', '32182'] )

# Set Target Names
target_list.append( 'Trent Williams' )
#target_list.append( '98 UL/PF Ed Reed' )
target_id_list.append( '34091' )
#target_id_list.append( '32175' )

# what number test to run?
# 0 - Trent Williams Replay Set, 1 - Ed Reed UL Set
h=0

player_names = player_name_list[h]
player_ids = player_id_list[h]
target = target_list[h]
target_id = target_id_list[h]

In [107]:
player_names[0]
player_ids[0]

'34093'

### use only when necessary, refreshes all player data from site

In [44]:
import time

link_start = 'https://www.muthead.com/18/players/prices/'
link_end = '/playstation-4/'
link_end_ref = '/playstation-4/refresh/'
start = time.time()
print("timestart")

for k in range(0, len(player_ids)):
    linky = link_start + player_ids[k] + link_end_ref
    browser.get(linky)
    time.sleep(1)
    #browser.quit()  # uncomment to close browser but will log you out of site

end = time.time()
print('Time in seconds for completion:\t', end - start)

timestart
Time in seconds for completion:	 25.622355937957764


In [113]:
import time
import pandas as pd


def start_up():
    
    start = time.time() # INITIALIZE TIMER FOR COMPUTING TIME
    print("timestart")

    link_start = 'https://www.muthead.com/18/players/prices/' # LINK PREFIX/POSTFIX
    link_end = '/playstation-4/'

    min_prices = [] # LIST FOR LOWEST BUYITNOW PRICE PER PLAYER
    N = len(player_ids)

    for i in range(0,N):
    
        print('startup.... i= ', i)
        
        mut_url = link_start + player_ids[i] + link_end
        print(mut_url)
        try:
            r = urllib.request.urlopen(mut_url).read()
    
            soup = BeautifulSoup(r, 'lxml')
    
            tables = soup.select('table')
    
            print('\n= = = = = ', player_names[i], '= = = = = ')
    
            print('retrieving prices')
            df = retrieve_prices(tables[0])
            time.sleep(2) 
    
            print('retrieving stats')
            df2 = retrieve_stats(tables[2])
            time.sleep(2)
    
            print()
            current_min = df['Buy Now Price'][0].replace(",", "")
            print('cur min for: ', player_names[i], ' ', player_ids[i], ' ', current_min, '')
            lower_q = df2['Price'][5].replace(",", "")
    
    # potential deal
    
    #if ( int(current_min) < int(lower_q) ):     # add back , curr out from string . to float err
        #print ('======DEAL======')
        
            print ('Current Min:\t', current_min)
            #print ('Avg Price:\t', avg)
            print ('Q1 Price:\t', lower_q)
            min_prices.append(current_min)
    
            temp = time.time()
            print('Time elapsed:\t', temp - start)
    

        except requests.ConnectionError as e:
            print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
            print(str(e))
        except requests.Timeout as e:
            print("OOPS!! Timeout Error")
            print(str(e))
        except requests.RequestException as e:
            print("OOPS!! General Error")
            print(str(e))
        except KeyboardInterrupt:
            print("Someone closed the program")
        
    end = time.time()
    print('Time in seconds for completion:\t', end - start)
    print(min_prices)
    return min_prices

In [114]:
min_prices = start_up()
#min_prices
#put_together()

timestart
startup.... i=  0
https://www.muthead.com/18/players/prices/34093/playstation-4/

= = = = =  Andy Dalton = = = = = 
retrieving prices
retrieving stats

cur min for:  Andy Dalton   34093   100000 
Current Min:	 100000
Q1 Price:	 <td class="value">82500</td>
Time elapsed:	 4.67718505859375
startup.... i=  1
https://www.muthead.com/18/players/prices/34094/playstation-4/

= = = = =  Laremy Tunsil = = = = = 
retrieving prices
retrieving stats

cur min for:  Laremy Tunsil   34094   68000 
Current Min:	 68000
Q1 Price:	 <td class="value">72000</td>
Time elapsed:	 10.115678071975708
startup.... i=  2
https://www.muthead.com/18/players/prices/34095/playstation-4/

= = = = =  Lamer Miller = = = = = 
retrieving prices
retrieving stats

cur min for:  Lamer Miller   34095   75000 
Current Min:	 75000
Q1 Price:	 <td class="value">60000</td>
Time elapsed:	 14.623569011688232
startup.... i=  3
https://www.muthead.com/18/players/prices/34096/playstation-4/

= = = = =  Eric Decker = = = = = 
r

In [112]:
min_prices

['441000', '340000', '91000', '55000', '45750']

In [104]:
df2

,Q,Price
0,<td>Average (Mean)</td>,"<td class=""value"">909,914</td>"
1,<td>Q1 - Q3 Range</td>,"<td class=""value"">885,000 - 937,500</td>"
2,<td>Maximum</td>,"<td class=""value"">1,135,000</td>"
3,<td>Upper Quartile (Q3)</td>,"<td class=""value"">937,500</td>"
4,<td>Median (Q2)</td>,"<td class=""value"">900,000</td>"
5,<td>Lower Quartile (Q1)</td>,"<td class=""value"">885,000</td>"
6,<td>Minimum</td>,"<td class=""value"">780,000</td>"


In [105]:
df3

,Time Sold,Price
0,"Mar 22, 2018/abbr",880000
1,"Mar 22, 2018/abbr",875000
2,"Mar 21, 2018/abbr",950000
3,"Mar 21, 2018/abbr",905000
4,"Mar 21, 2018/abbr",937500
5,"Mar 21, 2018/abbr",885000
6,"Mar 21, 2018/abbr",910000
7,"Mar 21, 2018/abbr",895000
8,"Mar 21, 2018/abbr",890000


In [73]:
def put_together():
    start = time.time()

    summ = 0
    for i in range(0,len(player_names)):
        summ+= int(min_prices[i])
    
    print('Cost to Build:\t', summ)

    mut_url = link_start + target_id + link_end
    r = urllib.request.urlopen(mut_url).read()
    soup = BeautifulSoup(r,"lxml")
    tables = soup.select('table')
    time.sleep(1) 

    print('retrieving prices')
    df = retrieve_prices(tables[0])
    time.sleep(1) 

    print('retrieving stats')
    df2 = retrieve_stats(tables[2])
    print()

    df3 = retrieve_sales(tables[1])

    current_min = df['Buy Now Price'][0].replace(",", "")
    lower_q = df2['Price'][5].replace(",", "")
    last_x_mean = df3['Price'].mean()
       
    cur_min = float (current_min )
    after_tax = cur_min*0.9

    if ( after_tax > summ):
        print('==+++===== BUY! BUY! BUY! =====+++==\n')
    else:
        print('X X X X X X NOT WORTH IT! X X X X X X\n')
    
    for i in range(0, len(player_names)):
        print( player_names[i] + '\t' + min_prices[i] )

    print ('\nCost to build:\t\t', summ)
    print ('Lowest ' + target +':\t', int(cur_min))
    print ('After Tax :\t\t', int(after_tax))
    print ('Last x mean:\t\t', int(last_x_mean) )
    print('Potential profit:\t', int(after_tax-summ))

    end = time.time()
    timey = end - start
    timey = str(round(timey, 2))

    print('\nTime in seconds for completion:\t', timey)


In [41]:
# todo: code to save into dataframe (long term)


In [ ]:
'''
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

## do for each player

r = requests.get(url, headers=headers)
# make sure that the page exist
if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('h1')
    if title is not None:
        title_text = title.text.strip()
        print(title_text)

tables = soup.select('table')
'''

###### Matthew Johnson, March 21, 2018